# Análise dos Tópicos 

Esse notebook possui os códigos para a análise dos tópicos extraídos dos diários oficiais. 

## Imports Necessários

In [1]:
# Código para poder usar os módulos implementados nesse repositório no notebook 
import sys, os
path_module = os.path.abspath('../modules/')
if path_module not in sys.path:
    sys.path.append(path_module)

In [2]:
# imports do projeto

from utils.process_gazette import ProcessGazette
from preprocess.pre_process_text import PreProcessText
from nlp.extract_topics import ExtractTopics

KeyboardInterrupt: 

## Extração e análise dos tópicos do Diário Teste

In [ ]:
pp = ProcessGazette(BASE_DIR="gazettes/")
txt_files = [f for f in os.listdir("gazettes/") if f.endswith('.txt')]
all_pages = []
for txt_file in txt_files:
    pages = pp.break_pages(txt_file, "ANO [X|V|I]+ ", save_file=True)
    all_pages.append(pages)

In [ ]:
all_topics = {}
topics_ = {}
ppt = PreProcessText("pt_core_news_lg")
for i in range(len(all_pages)):
    all_topics[i] = {}
    for page, text in all_pages[i].items():
        extractor = ExtractTopics(model = 'default')
        topics_[page] = {'text': text, 'topics': []}
        tokens_ = ppt.process_text(topics_[page]['text'])        
        try:
            if len(tokens_) > 5 and page:
                topics_[page]['tokens'] = tokens_
                topics_[page]['topics'], topics_[page]['topics_info'] = extractor.extract_topics(topics_[page]['tokens'])
                all_topics[i][page] = topics_[page]
        except Exception as e:
            print(page)
            print(tokens_)
            print(topics_[page]['text'])
            print(topics_[page]['tokens'])
            print(f"Error: {e}")


In [ ]:
all_topics[1][2]['topics_info']

,Topic,Count,Name,Representation,Representative_Docs
0,-1,67,-1_exceto_infracao_moreira_servico,"[exceto, infracao, moreira, servico, infrator,...","[exceto, sexta-feira, sexta-feira]"
1,0,26,0_municipal_urbanismo__,"[municipal, urbanismo, , , , , , , , ]","[municipal, municipal, municipal]"
2,1,25,1_disposto_cumprimento_dantas_gestao,"[disposto, cumprimento, dantas, gestao, entrar...","[cumprimento, disposto, cumprimento]"
3,2,24,2_sancao_cassacao_vinicius_magalhaes,"[sancao, cassacao, vinicius, magalhaes, couber...","[sancao, cassacao, sancao]"
4,3,19,3_pereira_claudio_antonio_andrey,"[pereira, claudio, antonio, andrey, salvador, ...","[Antonio, Andrey, Pereira]"
5,4,19,4_funcionamento_aplicavel_solicitar_medida,"[funcionamento, aplicavel, solicitar, medida, ...","[funcionamento, funcionamento, funcionamento]"
6,5,17,5_secretario_secretaria__,"[secretario, secretaria, , , , , , , , ]","[secretario, secretario, secretario]"
7,6,16,6_decreto_penalidade_licenca_legislacao,"[decreto, penalidade, licenca, legislacao, jur...","[decreto, decreto, decreto]"
8,7,14,7_educacao_ensino_classe_juventude,"[educacao, ensino, classe, juventude, junior, ...","[Educacao, Educacao, educacao]"
9,8,11,8_gabinete_oficial_unidade_registrar,"[gabinete, oficial, unidade, registrar, entida...","[gabinete, gabinete, gabinete]"
